In [1]:
%load_ext autoreload
%autoreload 2

## let's attack our house-prices example

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
train = pd.read_csv('../data/train.csv', sep=",")
test = pd.read_csv('../data/test.csv')

import sklearn
y = train['SalePrice']
X = train.drop('SalePrice', axis=1)
categorical = [var for var in X.columns if X[var].dtype=='O']
numerical = [var for var in X.columns if X[var].dtype!='O']
X[categorical] = X[categorical].fillna('None')

# auto-sklearn can not deal with categorical variables
X= pd.concat([pd.get_dummies(X[categorical], dummy_na=True), X[numerical]], axis=1)

y = np.log1p(y)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, random_state=42, test_size=0.2)

# Auto-Sklearn

[install auto-sklearn](https://automl.github.io/auto-sklearn/master/installation.html)

In [4]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
# ! cat auto-sklearn-requirements.txt | xargs -n 1 -L 1 pip3 install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   209  100   209    0     0    484      0 --:--:-- --:--:-- --:--:--   488


In [5]:
!pip install -U auto-sklearn==0.12.4

In [6]:
!pip install -U scikit-learn==0.24.0

     |████████████████████████████████| 22.3 MB 8.7 MB/s eta 0:00:01     |████████████████████▉           | 14.5 MB 4.8 MB/s eta 0:00:02     |███████████████████████         | 16.1 MB 4.6 MB/s eta 0:00:02
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-tabular 0.0.16b20210114 requires scikit-learn<0.24,>=0.22.0, but you have scikit-learn 0.24.0 which is incompatible.
autogluon-core 0.0.16b20210114 requires scikit-learn<0.24,>=0.22.0, but you have scikit-learn 0.24.0 which is incompatible.


In [6]:
import sklearn.metrics
import autosklearn.regression
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("default")

[Parameters](https://automl.github.io/auto-sklearn/master/api.html#regression)

In [7]:
! rm -rf /tmp/autosklearn_*
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=600,
    per_run_time_limit=30,
    memory_limit = 4096,
    ensemble_size = 8, 
    ensemble_nbest=4,
    max_models_on_disc = 16,
    n_jobs = 2,
    include_estimators = ['gradient_boosting', 'ard_regression', 'sgd'],
    resampling_strategy = 'cv',
    # include_preprocessors=["no_preprocessing"],
    tmp_folder='/tmp/autosklearn_regression_example_tmp',
    output_folder='/tmp/autosklearn_regression_example_out',
    delete_tmp_folder_after_terminate = True,
    delete_output_folder_after_terminate = False
)

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
automl.fit(X_train, y_train, dataset_name='house-prices')

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/autosklearn/automl.py:820: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif X.dtype in (np.float64, np.float):
/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/autosklearn/automl.py:524: DeprecationWarning: dist() and linux_distribution() functions are deprecated in Python 3.5
  self._logger.debug('  Distribution: %s', platform.linux_distribution())
/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/autosklearn/pipeline/base.py:327: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this wi

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/autosklearn/pipeline/components/regression/gradient_boosting.py:159: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name="min_samples_leaf", lower=1, upper=200, default_value=20, log=True)
/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/autosklearn/pipeline/components/regression/gradient_boosting.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is 

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/autosklearn/metalearning/metalearning/meta_base.py:50: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (Configuration(configuration_space, values=configuration))
/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/autosklearn/metalearning/metalearning/meta_base.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the

AutoSklearnRegressor(delete_output_folder_after_terminate=False,
                     ensemble_nbest=4, ensemble_size=8,
                     include_estimators=['gradient_boosting', 'ard_regression',
                                         'sgd'],
                     max_models_on_disc=16, memory_limit=4096, n_jobs=2,
                     output_folder='/tmp/autosklearn_regression_example_out',
                     per_run_time_limit=30, resampling_strategy='cv',
                     time_left_for_this_task=600,
                     tmp_folder='/tmp/autosklearn_regression_example_tmp')

In [9]:
print(automl.show_models())

autosklearn.__version__

[(0.625000, SimpleRegressionPipeline({'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler', 'feature_preprocessor:__choice__': 'select_percentile_regression', 'regressor:__choice__': 'ard_regression', 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_max': 0.7760749320397202, 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_min': 0.28724945996048523, 'feature_preprocessor:select_percentile_regression:percentile': 50.0, 'feature_preprocessor:select_percentile_regression:score_func': 'mutual_info', 'regressor:ard_regression:alpha_1': 1e-06, 'regressor:ard_regression:alpha_2': 1.0000000000000004e-06, 'regressor:ard_regression:fit_intercept': 'True', 'regressor:ard

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'0.12.4'

In [10]:
predictions = automl.predict(X_test)
print("R2 score:", sklearn.metrics.r2_score(y_test, predictions))
print("mean-squared-error:", sklearn.metrics.mean_squared_error(y_test, predictions, squared=False))

R2 score: 0.8944212884104441
mean-squared-error: 0.14036433928773803


# autogluon.tabular

In [11]:
!pip install mxnet==1.7.0.post1

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
!pip install autogluon-core==0.0.16b20210114 autogluon-tabular==0.0.16b20210114

     |████████████████████████████████| 6.8 MB 1.2 MB/s eta 0:00:01     |███████████████████████████████▌| 6.7 MB 1.3 MB/s eta 0:00:01


  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.0
    Uninstalling scikit-learn-0.24.0:
      Successfully uninstalled scikit-learn-0.24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.0 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.
auto-sklearn 0.12.4 requires scikit-learn<0.25.0,>=0.24.0, but you have scikit-learn 0.23.2 which is incompatible.


In [13]:
from autogluon.tabular import TabularPrediction as task
label_column = 'class'
X_train = pd.concat([X_train, y_train], axis=1)

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/autograd/numpy/numpy_wrapper.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  int_types = {_np.int, _np.int8, _np.int16, _np.int32, _np.int64, _np.integer}
/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/mxnet/numpy/utils.py:37: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_

In [14]:
predictor = task.fit(train_data=X_train, label='SalePrice',
                    time_limits=600, output_directory='/tmp/autogluon_regression_example_tmp',
                    presets='best_quality', problem_type='regression',
                    eval_metric='root_mean_squared_error', auto_stack=True, save_space=True,
                    cache_data=False,
                    excluded_model_types = ['XT', 'NN', 'FASTAI'],
                    nthreads_per_trial=2)
y_pred = predictor.predict(X_test)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred)
predictor.fit_summary()

[WARNING] [2021-07-08 08:44:15,467:root] WARNING: Using deprecated argument `time_limits`. Use `time_limit` instead! This will cause an exception in future releases.
[WARNING] [2021-07-08 08:44:15,492:root] Warning: `cache_data=False` will disable or limit advanced functionality after training such as feature importance calculations. It is recommended to set `cache_data=True` unless you explicitly wish to not have the data saved to disk.
[WARNING] [2021-07-08 08:44:15,496:autogluon.tabular.learner.abstract_learner] Warning: `cache_data=False` will disable or limit advanced functionality after training such as feature importance calculations. It is recommended to set `cache_data=True` unless you explicitly wish to not have the data saved to disk.


/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[1000]	train_set's rmse: 0.00456947	valid_set's rmse: 0.133531
[2000]	train_set's rmse: 0.0010747	valid_set's rmse: 0.133358
[1000]	train_set's rmse: 0.00732903	valid_set's rmse: 0.165941
[2000]	train_set's rmse: 0.00174502	valid_set's rmse: 0.163439
[3000]	train_set's rmse: 0.000438622	valid_set's rmse: 0.162985
[4000]	train_set's rmse: 0.000112624	valid_set's rmse: 0.162895
[5000]	train_set's rmse: 2.95958e-05	valid_set's rmse: 0.162879
[6000]	train_set's rmse: 7.70912e-06	valid_set's rmse: 0.162875
[7000]	train_set's rmse: 2.05131e-06	valid_set's rmse: 0.162874
[8000]	train_set's rmse: 5.91651e-07	valid_set's rmse: 0.162873
[9000]	train_set's rmse: 1.94868e-07	valid_set's rmse: 0.162873
[10000]	train_set's rmse: 6.65949e-08	valid_set's rmse: 0.162873
[1000]	train_set's rmse: 0.00491533	valid_set's rmse: 0.122266
[2000]	train_set's rmse: 0.00101376	valid_set's rmse: 0.119548
[3000]	train_set's rmse: 0.00023835	valid_set's rmse: 0.118962
[4000]	train_set's rmse: 5.69615e-05	valid_set'

/home/martin/miniconda3/envs/autoML/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


[1000]	train_set's rmse: 0.000442992	valid_set's rmse: 0.179714
[2000]	train_set's rmse: 1.36068e-05	valid_set's rmse: 0.179674
[1000]	train_set's rmse: 0.0052852	valid_set's rmse: 0.153197
[2000]	train_set's rmse: 0.000943732	valid_set's rmse: 0.152956
[3000]	train_set's rmse: 0.000216624	valid_set's rmse: 0.152796
[4000]	train_set's rmse: 6.47335e-05	valid_set's rmse: 0.152741
[5000]	train_set's rmse: 2.42404e-05	valid_set's rmse: 0.152729
[6000]	train_set's rmse: 8.29404e-06	valid_set's rmse: 0.152727
[7000]	train_set's rmse: 3.20374e-06	valid_set's rmse: 0.152727
[8000]	train_set's rmse: 1.1156e-06	valid_set's rmse: 0.152726
[9000]	train_set's rmse: 3.32339e-07	valid_set's rmse: 0.152726
[1000]	train_set's rmse: 0.00837283	valid_set's rmse: 0.150451
[2000]	train_set's rmse: 0.0018469	valid_set's rmse: 0.148982
[3000]	train_set's rmse: 0.000455694	valid_set's rmse: 0.148602
[4000]	train_set's rmse: 0.000124136	valid_set's rmse: 0.148497
[5000]	train_set's rmse: 3.47122e-05	valid_set

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/autogluon/core/utils/plots.py:141: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'RandomForestMSE_BAG_L0': 'StackerEnsembleModel_RF',
  'KNeighborsUnif_BAG_L0': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L0': 'StackerEnsembleModel_KNN',
  'LightGBM_BAG_L0': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L0': 'StackerEnsembleModel_LGB',
  'CatBoost_BAG_L0': 'StackerEnsembleModel_CatBoost',
  'XGBoost_BAG_L0': 'StackerEnsembleModel_XGBoost',
  'LightGBMCustom_BAG_L0': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L1': 'WeightedEnsembleModel',
  'RandomForestMSE_BAG_L1': 'StackerEnsembleModel_RF',
  'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'LightGBMCustom_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'RandomFo

In [15]:
perf

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.13033138813696107

# Scikit-Optimize

Probably you'll have to reload the notebook for the changes being in place. Scikit-Optimize works only with sklearn 0.23.2

In [16]:
!pip install scikit-learn==0.23.2

In [3]:
import skopt
import importlib
display(skopt.__version__)
importlib.reload(sklearn)
display(sklearn.__version__)

'0.8.1'

'0.23.2'

Since BayesSearchCV can not deal with missing values, we have to impute them before:

In [25]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
numeric_transformer = SimpleImputer(missing_values=np.nan, strategy='mean')

X_train.loc[:, numerical] = numeric_transformer.fit_transform(X_train.loc[:, numerical])
X_test.loc[:, numerical] = numeric_transformer.fit_transform(X_test.loc[:, numerical])

/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/home/martin/miniconda3/envs/autoML/lib/python3.7/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [13]:
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV
from sklearn.ensemble import GradientBoostingRegressor

regressor = BayesSearchCV(
    estimator = GradientBoostingRegressor(),
      search_spaces ={
         'learning_rate': Real(0.1,0.3),
         'loss': Categorical(['lad','ls','huber','quantile']),
         'max_depth': Integer(3,6),
      },
    n_iter=32,
    random_state=0,
    verbose=1, iid=True,
    cv=5, n_jobs=-1
  )
regressor.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished


BayesSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_iter=32, n_jobs=-1,
              random_state=0,
              search_spaces={'learning_rate': Real(low=0.1, high=0.3, prior='uniform', transform='identity'),
                             'loss': Categorical(categories=('lad', 'ls', 'huber', 'quantile'), prior=None),
                             'max_depth': Integer(low=3, high=6, prior='uniform', transform='identity')},
              verbose=1)

In [26]:
predictions = regressor.predict(X_test)
print("mean-squared-error:", sklearn.metrics.mean_squared_error(y_test, predictions, squared=False))

mean-squared-error: 0.136962734227094


# TPOT (Tree-based Pipeline Optimization Tool)

In [27]:
!pip install tpot

the steps below require a new-start of the notebook - if not the changes are not in place

In [28]:
!pip install --upgrade ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [29]:
from tpot import TPOTRegressor
from sklearn.datasets import load_digits 
from sklearn.model_selection import train_test_split 

tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42, n_jobs=-1)
tpot.fit(X_train, y_train)
print(np.sqrt(-tpot.score(X_test, y_test)))

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.017092608323869624

Generation 2 - Current best internal CV score: -0.017092608323869624

Generation 3 - Current best internal CV score: -0.017092608323869624

Generation 4 - Current best internal CV score: -0.016532793401539948

Generation 5 - Current best internal CV score: -0.016321850860696603

Best pipeline: GradientBoostingRegressor(RidgeCV(input_matrix), alpha=0.9, learning_rate=0.1, loss=ls, max_depth=10, max_features=0.4, min_samples_leaf=18, min_samples_split=15, n_estimators=100, subsample=0.6000000000000001)
0.13685696531984778
